In [1]:
import numpy as np
import strawberryfields as sf
import tensorflow as tf
import random

In [2]:
tf_displacement_magnitude = tf.Variable(0.1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
steps = 500

batch_size = 10
threshold = 0.5

for step in range(steps):
    batch = [1 if random.random() > threshold else -1 for _ in range(batch_size)]
    alpha_val = 0.7 * np.array(batch)
    
    p_1 = batch.count(1)/batch_size
    p_0 = 1 - p_1
        
    eng = sf.Engine(backend="tf", backend_options={
        "cutoff_dim": 7,
        "batch_size": batch_size,
    })
    
    circuit = sf.Program(1)

    displacement_magnitude = circuit.params("displacement_magnitude")
    
    alpha = circuit.params("alpha")

    with circuit.context as q:
        sf.ops.Dgate(alpha, 0.0) | q[0]
        sf.ops.Dgate(displacement_magnitude, 0.0) | q[0]
        
    with tf.GradientTape() as tape:
        results = eng.run(circuit, args={
            "displacement_magnitude": tf_displacement_magnitude,
            "alpha": alpha_val
        })
        
        # get the probability of |0>
        p_zero = results.state.fock_prob([0])
        
        # get the porbability of anything by |0>
        p_one = 1 - p_zero
        
        loss = 0
        
        # p(a|1) = [p(1|a) p(1)] / p(a)
        # p(-a|0) = [p(0|-a) p(0)] / p(-a)
        
        for i, mult in enumerate(batch):
            if mult == 1:
                loss += - (p_one[i] * p_1) / 2
            else:
                loss += - (p_zero[i] * p_0) / 2
                
        loss /= batch_size
        
    gradients = tape.gradient(loss, [tf_displacement_magnitude])
    opt.apply_gradients(zip(gradients, [tf_displacement_magnitude]))
    
    if (step + 1) % 10 == 0:
        print("Learned displacement value at step {}: {}".format(step+1, tf_displacement_magnitude.numpy()))

Learned displacement value at step 10: 0.19980643689632416
Learned displacement value at step 20: 0.2970784306526184
Learned displacement value at step 30: 0.3914494812488556
Learned displacement value at step 40: 0.4819519817829132
Learned displacement value at step 50: 0.5624663829803467
Learned displacement value at step 60: 0.6297617554664612
Learned displacement value at step 70: 0.6830540299415588
Learned displacement value at step 80: 0.7237957119941711
Learned displacement value at step 90: 0.7661814093589783
Learned displacement value at step 100: 0.802850604057312
Learned displacement value at step 110: 0.8245241045951843
Learned displacement value at step 120: 0.8384523987770081
Learned displacement value at step 130: 0.8384208083152771
Learned displacement value at step 140: 0.8375372886657715
Learned displacement value at step 150: 0.8455305695533752
Learned displacement value at step 160: 0.843255341053009
Learned displacement value at step 170: 0.8340648412704468
Learned

In [17]:
def raise_exception(value_name: str):
    raise ValueError(f'{value_name} parameter value must be set')

In [18]:
{"a":3}.get("b", raise_exception('b'))

ValueError: b parameter value must be set